In [513]:
from tools import analysistools as atools
import nanoparticle
from nanoparticle import Ligand, NanoParticle
import math
from membranesimulation import MembraneSimulation
import numpy as np
from tools import icosatiler
from tools import vectools
from tools import misctools
import k3d
import os
import random
from numpy import pi, cos, sin, arccos, arange

In [408]:
mag = lambda v : np.sqrt(np.sum([i*i for i in v]))

In [522]:
def goldenSpiral(n,rad):
    num_pts = 60
    indices = arange(0, n, dtype=float) + 0.5
    phi = arccos(1 - 2*indices/n)
    theta = pi * (1 + 5**0.5) * indices
    x, y, z = cos(theta) * sin(phi), sin(theta) * sin(phi), cos(phi)
    sphs = []
    for i in range(len(x)):
        spp = icosatiler.crt2SphPol((x[i],y[i],z[i]))
        spp = (rad,spp[1],spp[2])
        sphs.append(spp)
    return sphs
    

In [457]:
def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

In [410]:
def norm(v):
    m = mag(v)
    vN = [float(i)/float(m) for i in v]
    return vN

In [385]:
def crossProd(u,v):
    return ((u[1]*v[2]-u[2]*v[1]),(u[2]*v[0]-u[0]*v[2]),(u[0]*v[1]-u[1]*v[0]))

In [386]:
def makeParticleModelFromPhenome(particle):
    points = []
    plot = k3d.plot()
    i = 1
    maxEps = 0
    for l in particle.ligands:
        if l.eps>maxEps:
            maxEps = l.eps
    for l in particle.ligands:
        if l.eps > 0.0:
            lx = l.rad*np.sin(l.polAng)*np.cos(l.aziAng)
            ly = l.rad*np.sin(l.polAng)*np.sin(l.aziAng)
            lz = l.rad*np.cos(l.polAng)
            lc = '0x' + ('#%02x%02x%02x' % (0, int((l.eps/maxEps)*255.0), 0))[1:]
            
            plot += k3d.points([lx,ly,lz],point_size=1,color=int(lc, 16))
            #plot += k3d.points([lx,ly,lz],point_size=1,color=int(0x00ffff*(l.eps/maxEps)))
            plot += k3d.text(str(i),[lx,ly,lz],color=0xffffff)
            i+=1
    plot += k3d.points([0,0,0],point_size=7,color=0xff0000)
    return plot

In [387]:
def buildCoveredParticle(eps,rad):
    particle = NanoParticle()
    lPos = icosatiler.cover72SpherePolar(rad)
    for v in lPos:
        particle.addLigand(Ligand(eps,1,v[0],v[1],v[2]))
    return particle

In [388]:
def spaceIsOccupied(particle,v):
    for l in particle.ligands:
        d = atools.greatArcDist((l.polAng,l.aziAng),(v[1],v[2]),particle.sig)
        if abs(d) < l.size:
            return True
    return False

In [475]:
def getClosestIcos(particle,v):
    lPos = icosatiler.cover72SpherePolar(particle.sig)
    closest = v
    minD = 1e8
    for l in lPos:
        v1 = icosatiler.sphPol2Crt(l) 
        v2 = icosatiler.sphPol2Crt(v)
        d = np.sum([j*j for j in np.subtract(v1,v2)])
        if abs(d) < minD:
            minD = d
            closest = l
    return closest

In [390]:
# lPos = icosatiler.cover72SpherePolar(4)
# dists = []
# minD = 1e8
# for v in lPos:
#     for l in lPos:
#         if l != v:
#             v1 = icosatiler.sphPol2Crt(l) 
#             v2 = icosatiler.sphPol2Crt(v)
#             d = np.sum([j*j for j in np.subtract(v1,v2)])
#             if abs(d) < minD:
#                 minD = d
#     dists.append(np.sqrt(minD))
# print np.mean(dists)

In [391]:
def addLigand(particle,eps,v,lock=False):
    if lock:
        v = getClosestIcos(particle,v)
    if not spaceIsOccupied(particle,v):
        particle.addLigand(Ligand(eps,1,v[0],v[1],v[2]))
    return particle

In [392]:
def addBelt(particle,eps,rad,n,phi,lock=False):
    step = np.pi*2/float(n)
    for i in range(n):
        v = (rad,step*i,phi)
        particle = addLigand(particle,eps,v,lock)
    return particle

In [393]:
def getArcAngle(dist,rad):
    return dist/rad

In [426]:
def addRing(particle,eps,rad,pos,n,spacing,lock=False):
    step = np.pi*2/float(n)
    if lock:
        origin = getClosestIcos(particle,(rad,pos[0],pos[1]))
        pos = (origin[1],origin[2])
    for i in range(n):
        tD = getArcAngle(spacing,rad)
        tP = step*i
        v = (rad,tD,tP)
        vC = icosatiler.sphPol2Crt(v)
        v1 = icosatiler.sphPol2Crt((rad,0,0))
        v2 = icosatiler.sphPol2Crt((rad,pos[0],pos[1]))
        v3 = crossProd(v1,v2)
        if np.sum([j*j for j in v3])== 0.0:
            #it's parallel!
            particle = addLigand(particle,eps,v,lock)
            continue
        dp = np.dot(v1,v2)
        #the magic sauce :)
        ang = np.arctan2(mag(np.cross(v1,v2)), np.dot(v1,v2))
        rM = vectools.buildERMatrix(v3, ang)
        vC = np.dot(rM,vC)
        v = icosatiler.crt2SphPol(vC)
        particle = addLigand(particle,eps,v,lock)
    return particle

In [492]:
def addPatch(particle,eps,rad,pos,n,spacing,lock=False):
    if lock:
        origin = getClosestIcos(particle,(rad,pos[0],pos[1]))
        pos = (origin[1],origin[2])
    particle = addLigand(particle,eps,(rad,pos[0],pos[1]),lock)
    particle = addRing(particle,eps,rad,pos,n,spacing,lock)
    return particle

In [524]:
def addMesh(particle,eps,rad,n,lock=False):
    mesh = goldenSpiral(n,particle.sig)
    for point in mesh:
        particle = addLigand(particle,eps,point,lock)
    return particle

In [497]:
#makeParticleModelFromPhenome(buildCoveredParticle(6,4))

In [538]:
p = NanoParticle()
m = 3
step = np.pi*2/float(m)
p = addMesh(p,6,4,24,False)
# for i in range(m):
#     p = addPatch(p,6,4,(i*step,np.pi/2.0),6,1.6,lock=True)
# p = addPatch(p,6,4,(np.pi/2.0,0),6,1.6,lock=True)
# p = addPatch(p,6,4,(3*np.pi/2.0,0),6,1.6,lock=True)
# p = addBelt(p,6,4,29,0,lock=True)

In [539]:
makeParticleModelFromPhenome(p)

Plot(antialias=True, background_color=16777215, grid=[-1, -1, -1, 1, 1, 1], height=512, object_ids=[121033671440, 120969076624, 121033053008, 121033673232, 121033671120, 121033672784, 121033672272, 121033672528, 121033673424, 121033672144, 121033674576, 121033673104, 121033674640, 121033670736, 121033056144, 121033673168, 121033674064, 121033673808, 121033806736, 121033806352, 121033807312, 121033672336, 121033674320, 121033806480, 121033674000, 121033807376, 121033807632, 121033807824, 121033672720, 121033808336, 121033807952, 121033808208, 121033809232, 121033805904, 121033807056, 121033809104, 121033809296, 121033807696, 121033808016, 121033809744, 121033809168, 121033953488, 121033808656, 121033809680, 121033809552, 121033954448, 121033954384, 121033954512, 121032914576])

In [510]:
RUNTIME = 25000
TIMESTEP = 0.01
OUTDIR = "/Users/joelforster/Projects/optidb/models/out/"
RUNDIR = "/Users/joelforster/Projects/optidb/models/run/"
TEMPLATEDIR = "/Users/joelforster/Projects/optihedron/mem/template"
TEMPLATEDATAPATH = os.path.join(TEMPLATEDIR,'data.template')
TEMPLATEINPUTPATH = os.path.join(TEMPLATEDIR,'in.template')

ensure_dir(OUTDIR)
ensure_dir(RUNDIR)

sim = MembraneSimulation(
        'model_'+misctools.randomStr(10),
        p,
        RUNTIME,
        TIMESTEP,        
        OUTDIR,
        RUNDIR,
        TEMPLATEDATAPATH,
        TEMPLATEINPUTPATH,
        rAxis=vectools.randomUnitVector(),
        rAmount=random.uniform(0.3141,3.141)        
        )
sim.saveFiles()

saved files: /Users/joelforster/Projects/optidb/models/run/model_WHPC5DADA4_data.data, /Users/joelforster/Projects/optidb/models/run/model_WHPC5DADA4_script.in


[(4, 0.7227342478134157, -1.199981614864327),
 (4, 1.318116071652818, 2.6832404625866064),
 (4, 1.8234765819369751, 0.2832772328579506),
 (4, 2.4188584057763776, -2.1166859968707015)]